In [2]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt

# --------- settings (edit these) ----------
FS = 500  # Hz, set to your actual EMG sampling rate
INPUT_DIR = r"C:\Users\lilin\OneDrive\Desktop\Project\EMG\Raw"
OUTPUT_DIR = r"C:\Users\lilin\OneDrive\Desktop\Project\EMG\Filtered"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# --------- filters / envelope ----------
def bandpass_emg(emg, fs, low=20, high=240, order=4):
    nyq = 0.5 * fs
    b, a = butter(order, [low / nyq, high / nyq], btype="band")
    return filtfilt(b, a, emg)

def highpass_emg(emg, fs, cutoff=5, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype="highpass")
    return filtfilt(b, a, emg)

def lowpass_emg(emg, fs, cutoff=10, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype="lowpass")
    return filtfilt(b, a, emg)

def moving_rms(emg_rect, fs, window_ms=300):
    win = int(window_ms / 1000.0 * fs)
    win = max(win, 1)
    kernel = np.ones(win) / win
    sq = emg_rect ** 2
    return np.sqrt(np.convolve(sq, kernel, mode="same"))

# def process_emg_channel(emg_raw, fs):
#     emg_raw = np.asarray(emg_raw, dtype=float)
#     emg_raw = np.nan_to_num(emg_raw, nan=0.0)
#     emg_bp   = bandpass_emg(emg_raw, fs)
#     emg_rect = np.abs(emg_bp)
#     emg_env  = moving_rms(emg_rect, fs, window_ms=300)
#     return emg_env

def process_emg_channel(emg_raw, fs):
    emg_raw = np.asarray(emg_raw, dtype=float)
    emg_raw = np.nan_to_num(emg_raw, nan=0.0)
    emg_bp   = highpass_emg(emg_raw, fs)
    emg_rect = np.abs(emg_bp)
    emg_env  = lowpass_emg(emg_rect, fs)
    return emg_env

# --------- loop over files ----------
for path in glob.glob(os.path.join(INPUT_DIR, "*.csv")):
    df = pd.read_csv(path)

    cols_to_process = df.columns[10:26]

    for col in cols_to_process:
        # process both EMG_MVC_CHx and EMG_CHx the same way
        emg_processed = process_emg_channel(df[col].values, FS)
        # overwrite the original column with filtered/enveloped signal
        df[col] = emg_processed

    out_name = os.path.basename(path).replace(".csv", "_butter_low_high.csv")
    out_path = os.path.join(OUTPUT_DIR, out_name)
    df.to_csv(out_path, index=False)
    print(f"Saved: {out_path}")

Saved: C:\Users\lilin\OneDrive\Desktop\Project\EMG\Filtered\P3P4_EMG_butter_low_high.csv
